In [1]:
import torch
import os
import imageio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from scipy.ndimage import binary_erosion
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
import pandas as pd
from torchvision import datasets
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import WeightedRandomSampler


In [2]:
#images = "/home/bandyadkas/cellcyle/data/"
images = "/mnt/efs/woods_hole/bbbc_cellcycle/model_data/data/"

In [3]:
transform = {
        'train': transforms.Compose([
            #transforms.Resize([224,224]), # Resizing the image as the VGG only take 224 x 244 as input size
            #transforms.RandomHorizontalFlip(), # Flip the data horizontally
            #TODO if it is needed, add the random crop
            transforms.ToTensor(),
            #transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
        ]),
        'test': transforms.Compose([
            #transforms.Resize([224,224]),
            #transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            #transforms.Normalize(mean=(0), std=(1))
        ])
    }

In [4]:
all_images_raw = datasets.ImageFolder(images,transform=transform['train'])
print(len(all_images_raw))
#all_images = torchvision.datasets.DatasetFolder(
#    "/mnt/efs/woods_hole/bbbc_cellcycle/CellCycle/",
#    imageio.imread, ("merged.jpg"), transform=transform)
#print(len(all_images))

32266


In [5]:
import copy
print(all_images_raw.class_to_idx)
new_classIds = {'G1': 0, 'S': 1, 'G2': 2, 'Prophase': 3, 'Metaphase': 4, 'Anaphase': 5, 'Telophase': 6}
all_images_new_classIDs = copy.deepcopy(all_images_raw)
all_images_new_classIDs.class_to_idx = new_classIds
print(all_images_new_classIDs.class_to_idx)

{'Anaphase': 0, 'G1': 1, 'G2': 2, 'Metaphase': 3, 'Prophase': 4, 'S': 5, 'Telophase': 6}
{'G1': 0, 'S': 1, 'G2': 2, 'Prophase': 3, 'Metaphase': 4, 'Anaphase': 5, 'Telophase': 6}


In [6]:
sines = np.sin(2 * np.pi*np.array(all_images_new_classIDs.targets)/7)
cosines = np.cos(2 * np.pi*np.array(all_images_new_classIDs.targets)/7)

#print(np.shape(sines), np.shape(cosines))
sine_cosine_targets = np.stack((sines,cosines),axis=1)
#print(np.shape(sine_cosine_targets))


all_images_circ_targets = copy.deepcopy(all_images_new_classIDs)
all_images_circ_targets.targets = sine_cosine_targets

#print(np.shape(all_images_raw.targets))
#print(np.shape(all_images_new_classIDs.targets))
#print(np.shape(all_images_circ_targets.targets))
assert np.shape(all_images_circ_targets.targets)==(32266,2)


In [7]:
train_size = int(0.7 * len(all_images_circ_targets))
val_size = int(0.15 * len(all_images_circ_targets))
test_size = len(all_images_circ_targets) - (train_size + val_size)
print(train_size, val_size, test_size)
assert train_size + val_size + test_size == len(all_images_circ_targets)

22586 4839 4841


In [8]:
train_set, val_set, test_set = torch.utils.data.random_split(all_images_circ_targets, [train_size, val_size, test_size])
#train_set, val_set, test_set = torch.utils.data.random_split(all_images, [22538, 4829, 4831])

In [9]:
def _get_weights(subset,full_dataset):
    ys = np.array([y for _, y in subset])
    print(ys)
    counts = np.bincount(ys)
    label_weights = 1.0 / counts
    weights = label_weights[ys]

    print("Number of images per class:")
    for c, n, w in zip(full_dataset.classes, counts, label_weights):
        print(f"\t{c}:\tn={n}\tweight={w}")
        
    return weights


In [ ]:

train_weights = _get_weights(train_set,all_images_circ_targets)
train_sampler = WeightedRandomSampler(train_weights, len(train_weights))


In [ ]:
all_images_circ_targets.class_to_idx

In [ ]:
train_loader = DataLoader(train_set, batch_size=8, drop_last=True, sampler=train_sampler)
val_loader = DataLoader(val_set, batch_size=8 , drop_last=True, shuffle=True)
test_loader = DataLoader(test_set, batch_size=8, drop_last=True, shuffle=True)



In [ ]:
## Set up tensorboard
#writer = SummaryWriter('/mnt/efs/woods_hole/bbbc_cellcycle/classify_cellCycle_bandyadka/runs/cellcycle_resnet50_CyclicalFeaturesRegression_2')

writer = SummaryWriter('/home/bandyadkas/cellcyle/runs/cellcycle_resnet50_CyclicalFeaturesRegression_2')


In [ ]:
#!tensorboard --logdir=runs

In [ ]:
resnet50_model = torchvision.models.resnet50(pretrained = False, progress  = True, num_classes=2)
#loss_fn = nn.CrossEntropyLoss()


In [ ]:

from torchsummary import summary
#resnet50_model_regression = nn.Sequential(*list(resnet50_model.children())[:-1])
resnet50_model_regression = resnet50_model
#resnet50_model_regression.add_module(nn.Sigmoid)
summary(resnet50_model_regression.cuda(),(3,66,66))

loss_fn = nn.MSELoss()
optimizer = optim.Adam(resnet50_model_regression.parameters(), lr=0.001)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet50_model_regression.to(device)
print(f"Will use device {device} for training")

In [ ]:
from tqdm import tqdm

def train(model,loss,train_dataloader):
    model.train()
    epoch_loss = 0
    num_batches = 0
    for x, y in tqdm(train_dataloader):
            
        sines = [np.sin(2*np.pi*yi/7) for yi in np.array(y)]
        cosines = [np.cos(2*np.pi*yi/7) for yi in np.array(y)]
        sine_cosine_y = np.stack((sines,cosines),axis=-1)
        
        y = torch.from_numpy(sine_cosine_y)
        x, y = x.to(device), y.to(device)
        
        #print(x.shape, y.shape)
        optimizer.zero_grad()

        y_pred = model(x)
        l = loss(y_pred, y.float())
        l.backward()
        optimizer.step()

        epoch_loss += l
        num_batches += 1

    return epoch_loss/num_batches

def evaluate(model, loss, dataloader):
    
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in tqdm(dataloader):
            sines = [np.sin(2*np.pi*yi/7) for yi in np.array(y)]
            cosines = [np.cos(2*np.pi*yi/7) for yi in np.array(y)]
            sine_cosine_y = np.stack((sines,cosines),axis=-1)
            
            y = torch.from_numpy(sine_cosine_y)

            x, y = x.to(device), y.to(device)
            

            logits = model(x)
            
            val_loss = loss(logits,y.float())
            
            
            #probs = torch.nn.Softmax(dim=1)(logits)
            #predictions = torch.argmax(logits, dim=2)
            #predictions1 = torch.argmax(logits, dim=1)
            #print(np.shape(logits))
            #print(np.shape(y))
            #print(np.shape(predictions1))
            #print(np.shape(predictions))

            correct += int(torch.sum(logits == y).cpu().detach().numpy())
            total += len(y)

        accuracy = correct/total

    return accuracy, val_loss, logits 

def validate(model,loss, validation_dataloader):
    '''Evaluate prediction accuracy on the validation dataset.'''
    
    model.eval()
    return evaluate(model,loss,validation_dataloader)

def test(model,loss,test_dataloader):
    '''Evaluate prediction accuracy on the test dataset.'''
    
    model.eval() 
    return evaluate(model, loss,test_dataloader)

In [ ]:
step = 0
epochs = 12
for epoch in range(epochs+1):
    while step < epoch: 
    
        epoch_loss = train(resnet50_model_regression,loss_fn,train_loader)
        print(f"epoch {epoch}, training loss={epoch_loss}")
        
        logits = validate(resnet50_model, loss_fn,val_loader)
        print(logits)
        
    
        validation_accuracy, validation_loss, logits = validate(resnet50_model_regression, loss_fn,val_loader)
        print(f"epoch {epoch}, validation accuracy={validation_accuracy}")
    
        writer.add_scalar('Loss/train', epoch_loss.cpu().detach().numpy(),step)      
        writer.add_scalar('Accuracy/validation', validation_accuracy,step)
        writer.add_scalar('Loss/validation', validation_loss.cpu().detach().numpy(),step)
        
        while step < 51:
            state = {
                'epoch': epoch,
                'state_dict': resnet50_model_regression.state_dict(),
                'optimizer': optimizer.state_dict()
            }
            torch.save(state, "/home/bandyadkas/cellcyle/regressionmodel.pth")
        
        step += 1

In [ ]:
#load_model = torch.load("/mnt/efs/woods_hole/bbbc_cellcycle/classify_cellCycle_bandyadka/modelsave.pth")

In [ ]:
test_accuracy, test_loss = test(resnet50_model_regression,loss_fn,test_loader)
print(f"final test accuracy: {test_accuracy}")
writer.add_scalar('Accuracy/test', test_accuracy)
writer.add_scalar('Loss/test', test_loss.cpu().detach().numpy(),step)
        
